In [1]:
import torch  
import torch.nn as nn 
import torchvision
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [2]:
#Hyper parameters 
Epoch_num = 100
batch_size =64
learning_rate = 0.01

width = 224
height = 224

device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
training_transfrom = transforms.Compose([transforms.Resize((width,height)), transforms.RandomRotation(30),
                transforms.RandomVerticalFlip(),transforms.RandomHorizontalFlip(), 
                transforms.ToTensor() ,transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

training_data =  datasets.Flowers102(root='data', split='train', download= True, transform= training_transfrom)

training_loader = DataLoader(training_data, batch_size=batch_size, shuffle= True, num_workers=2)

val_transform = transforms.Compose([transforms.Resize((width,height)),transforms.ToTensor(), 
                                    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
val_data = datasets.Flowers102(root='data', split = 'val', download=True, transform = val_transform)

val_loader = DataLoader(val_data, batch_size= batch_size, shuffle=True)

test_transform = transforms.Compose([transforms.Resize((width,height)), transforms.ToTensor(), 
                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

test_data = datasets.Flowers102(root='data', split = 'test', download = True, transform = test_transform)
test_loader = DataLoader(test_data, batch_size= batch_size, shuffle = True)


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(5,5), stride = (1,1), padding=(1,1))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(5,5), stride = (1,1), padding=(1,1))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels= 128, kernel_size= (3,3), stride=(1,1), padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels = 128, out_channels= 128, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size= (2,2), stride= (2,2))
        self.fc1 = nn.Linear(128*27*27, 2048)
        self.fc2 = nn.Linear(2048, 2048)
        self.fc3 = nn.Linear(2048, 102)
        self.dropout = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # print(x.shape)
        # x = self.dropout(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # print(x.shape)
        x = F.relu(self.conv3(x))
        # x = self.pool(x)
        # print(x.shape)
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        # print(x.shape)
        x = x.view(-1, 128 *27 *27)
        # x = x.view(x.size(0), -1)
        x = self.dropout(self.fc1(x))
        x = self.dropout(self.fc2(x))
        # x = self.fc2(x)
        x = self.fc3(x)
        return x
        


model = CNN().to(device)

optimiser = optim.SGD(model.parameters(),lr=learning_rate, momentum=0.95)

criterion = nn.CrossEntropyLoss()

scheduler = optim.lr_scheduler.StepLR(optimiser, step_size= 30, gamma=0.5, verbose= True)

# x = torch.randn(0, 3, 240, 240)
# y = model(x)
# print(y.shape)
print(model)

Adjusting learning rate of group 0 to 1.0000e-02.
CNN(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=93312, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=102, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)


In [5]:
image, label = next(iter(training_loader))

image = image.to(device)

x = F.relu(model.conv1(image))
x = model.pool(x)
# print(x.shape)
x = F.relu(model.conv2(x))
x = model.pool(x)
# print(x.shape)
x = F.relu(model.conv3(x))
# x = model.pool(x)
# print(x.shape)
x = F.relu(model.conv4(x))
x = model.pool(x)
print(x.shape)


torch.Size([64, 128, 27, 27])


In [6]:
train_losses = []
valid_lossess = []

for epoch in range(Epoch_num):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0
    total = 0
    best_accuracy = 38

    
    model.train()
    for batch, (data,target) in enumerate(training_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        optimiser.zero_grad()
        out = model(data)
        loss = criterion(out, target)
        loss.backward()
        optimiser.step()
        train_loss += loss.item() * data.size(0)
        
        
    model.eval()
    with torch.no_grad():
      for data, target in val_loader:
        
          data = data.to(device)
          target = target.to(device)
        
          out = model(data)

          _, predicted = torch.max(out.data, 1)

          total += target.size(0)
          correct += (predicted == target).sum().item()
          loss = criterion(out, target)

          valid_loss += loss.item() * data.size(0)
    accuracy = (correct / total) *100
    
    if(accuracy > best_accuracy):
      best_accuracy = accuracy
      torch.save(model, 'model_best')
    scheduler.step()
    train_loss = train_loss/len(training_loader.sampler)
    valid_loss = valid_loss/len(val_loader.sampler)
    train_losses.append(train_loss)
    valid_lossess.append(valid_loss)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAccuracy: {:.6f}'.format(epoch, train_loss, valid_loss, accuracy))
    

def final_test(test_loader, model):
  with torch.no_grad():
    correct = 0 
    total = 0
    for data, target in test_loader:
      data = data.to(device)
      target = target.to(device)

      out = model(data)

      _, predicted = torch.max(out.data, 1)

      total += target.size(0)

      correct +=( predicted == target).sum().item()

    accuracy = (correct / total) *100
    print("Accuracy: {:.2f}".format(accuracy))

best_model = torch.load('model_best')
final_test(test_loader, best_model)


Adjusting learning rate of group 0 to 1.0000e-02.
Epoch: 0 	Training Loss: 4.627452 	Validation Loss: 4.619585 	Accuracy: 1.470588
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch: 1 	Training Loss: 4.611134 	Validation Loss: 4.573976 	Accuracy: 2.941176
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch: 2 	Training Loss: 4.433834 	Validation Loss: 4.212840 	Accuracy: 3.627451
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch: 3 	Training Loss: 4.226733 	Validation Loss: 4.068200 	Accuracy: 6.568627
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch: 4 	Training Loss: 4.001328 	Validation Loss: 3.897838 	Accuracy: 10.000000
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch: 5 	Training Loss: 3.914118 	Validation Loss: 4.135549 	Accuracy: 5.784314
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch: 6 	Training Loss: 3.805372 	Validation Loss: 3.992198 	Accuracy: 7.156863
Adjusting learning rate of group 0 to 1.0000e-02.
Epoch: 7 	Training Loss: 3.74587

In [ ]:
best_model = torch.load('model_best')
final_test(test_loader,best_model)